# Preprocessing

#### Tasks:

- Add title to article text

- Remove links, HTML, twitter usernames, emails

- Remove publisher signatures ( TRAIN ONLY )

#### Dependencies

In [2]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append('../src')

from datatasks.new_preprocess import preprocess

In [3]:
preprocess('../data/interim/')

Saving


In [3]:
DATA_INTERIM_PATH = '../data/interim/'

train = pd.read_csv(DATA_INTERIM_PATH + 'train_p.csv')
val = pd.read_csv(DATA_INTERIM_PATH + 'val_p.csv')

In [3]:
train.shape

(761639, 16)

In [4]:
val.shape

(200000, 15)

#### Add title to article text

In [7]:
def add_title_to_article_text(article):
    if article['title'] is not np.nan:
        return str(article['title']) + ' ' + str(article['article_text'])
    else:
        return str(article['article_text'])

In [8]:
for df in [train, val]:
    df['preprocessed_text'] = df.apply(add_title_to_article_text, axis=1)

#### Remove HTML artifacts

In [29]:
import re

def replace_html_stuff(article):
    preprocessed_text = article['preprocessed_text']
    
    # Remove all between left and right brackets
    preprocessed_text = re.sub(r'(?s)&lt.*?&gt;', '', preprocessed_text)
    preprocessed_text = re.sub(r'(?s)lt;.*?gt;', '', preprocessed_text)
    
    # Remove all html tags
    preprocessed_text = re.sub(r'&(.{1,5});', '', preprocessed_text)
    preprocessed_text = re.sub(r'amp;', '', preprocessed_text)
    
    # Remove !function
    preprocessed_text = re.sub(r'(?s)!function.*?(\?|\")\);', '', preprocessed_text)
    
    return preprocessed_text

In [30]:
for df in [train, val]:
    df['preprocessed_text'] = df.apply(replace_html_stuff, axis=1)

##### TODO: URLs, Twitter Usernames, Emails

In [38]:
train['domain'].value_counts().head(30)

foxbusiness                  115780
abqjournal                    91725
apnews                        56945
pri                           51773
newsline                      47590
counterpunch                  42407
motherjones                   38484
truthdig                      33677
dailywire                     20962
thedailybeast                 19477
poynter                       17274
baptistnews                   13700
nbcnews                       12827
reuters                       10914
washingtonblade                8256
consortiumnews                 7695
calwatchdog                    7610
thegoldwater                   6868
natmonitor                     6556
dissentmagazine                6292
politicalillusionsexposed      5664
chicagoreporter                5278
ivn                            4967
eppc                           4652
factcheck                      4485
mintpressnews                  4380
circa                          3608
billmoyers                  

In [5]:
from gensim.models.phrases import Phrases, Phraser
from nltk.tokenize.toktok import ToktokTokenizer

In [52]:
dailybeast = train[train['domain']=='circa'].reset_index()['preprocessed_text']

In [53]:
tokenizer = ToktokTokenizer()

In [54]:
all_tokens = [tokenizer.tokenize(dailybeast[i]) for i in range(len(dailybeast))]

In [57]:
phraser = Phraser(Phrases(all_tokens, min_count=len(dailybeast)/10, threshold=32))

In [58]:
phraser.phrasegrams

{(b'more', b'than'): (683, 45.714749252423864),
 (b'I', b'am'): (556, 36.06592330501416),
 (b'Associated', b'Press'): (603, 294.60565844885065),
 (b'Press', b'contributed'): (478, 163.766092521669),
 (b'New', b'York'): (635, 207.671408086006),
 (b'White', b'House'): (549, 154.56206005965092)}

In [63]:
check = train[train['preprocessed_text'].str.contains('\[(.*)\]')]

/Users/jmill/anaconda3/envs/SemEval2019-4/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [70]:
check.reset_index()['preprocessed_text'][5]

'John Lewis Advice for Young Activists: March\n I jumped in the car with two of the young people, and I drove to Selma, late at night, got there at three or four in the morning, got our sleeping bags out, got up the next morning, got dressed, went to the church, and we lined up and we marched. The rest is history. \nAs the 50th anniversary of the March on Washington for Jobs and Freedom approaches, the spirit of protest seems to be rekindling in many parts of the country. The Dream Defenders remain camped out in the Florida capitol, demanding justice for Trayvon Martin and an end to racial profiling; North Carolinas Moral Monday protesters have rallied and committed civil disobedience each week, defying the far-right agenda of their state legislators. In New York, the coalition against stop and frisk won its victory in court, but it took organizing, rallying, and yes, marching, to make it happen. \nIt is a good time, then, for a new memoir by congressman and 1960s civil rights leader J